In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsCogWriter

from src.run_task import get_grid, S1Processor

from dep_tools.searchers import PystacSearcher
from dep_tools.loaders import OdcLoader

from planetary_computer import sign_url

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask()
client.dashboard_link

In [ ]:
# Issues

# Study site configuration
region_code = "64,20"
datetime = "2023"
item_id = region_code

# And get the study site
grid = get_grid()
area = grid.loc[[(region_code)]]
area.explore()

In [ ]:
# Find some items
searcher = PystacSearcher(
    catalog="https://planetarycomputer.microsoft.com/api/stac/v1/",
    collections=["sentinel-1-rtc"],
    datetime=datetime,
    query = {"sat:orbit_state": {"eq": "descending"}}
)

items = searcher.search(area)

print(f"Found {len(items)} items")

In [ ]:
# Set up a data loader
loader = OdcLoader(
    crs=3832,
    resolution=100,
    bands=["vv", "vh"],
    groupby="solar_day",
    chunks=dict(time=1, x=4096, y=4096),
    fail_on_error=False,
    patch_url=sign_url
)

# Run the load process, which is lazy-loaded
input_data = loader.load(items, area)
input_data

In [ ]:
# Set up a data processor
processor = S1Processor()

# Plan the processing.
output_data = processor.process(input_data)
output_data

In [ ]:
output_data["count"].plot.imshow(size=10)

In [ ]:
output_data["mean_vv_vh"] = (output_data["mean_vv"] / output_data["mean_vh"])
output_data[["mean_vv", "mean_vh", "mean_vv_vh"]].to_array().plot.imshow(size=10, vmin=0, vmax=0.1)

In [ ]:
# Testing the Azure writer

# from dep_tools.writers import AzureDsWriter
# from dep_tools.namers import DepItemPath

# itempath = DepItemPath("geomad", "test", "0.0", datetime)

# writer = AzureDsWriter(
#     itempath=itempath,
#     overwrite=True,
#     convert_to_int16=False,
#     extra_attrs=dict(dep_version="0.0"),
# )

# writer.write(output_data, "test")


In [ ]:
# Testing the AWS writer

from dep_tools.writers import AwsDsCogWriter
from dep_tools.namers import DepItemPath

itempath = DepItemPath("geomad", "test", "0.0", datetime)

writer = AwsDsCogWriter(
    itempath=itempath,
    overwrite=False,
    convert_to_int16=False,
    extra_attrs=dict(dep_version="0.0"),
    bucket="files.auspatious.com"
)

writer.write(output_data, "test")

In [ ]:
from odc.stac import load
from pystac import Item

item = Item.from_file("https://deppcpublicstorage.blob.core.windows.net/output/dep_geomad_test/0-0/test/2023-01/dep_geomad_test_test_2023-01.stac-item.json")

data = load([item], chunks={})
data

In [ ]:
# This is the target path
dep_path = LocalPath(
    local_folder="data",
    sensor="s1",
    dataset_id="mosaic",
    version="0.0.0",
    time=datetime,
    zero_pad_numbers=True
)

# Set up a writer and write out the files
writer = LocalDsWriter(
    itempath=dep_path,
    output_nodata=0,
    use_odc_writer=True,
    overwrite=True,
    convert_to_int16=False
)
print(f"Writing to: {dep_path._folder(item_id)}")
out_files = writer.write(output_data, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()

In [ ]:
from pystac import Item
item = Item.from_file("https://s3.ap-southeast-2.amazonaws.com/files.auspatious.com/dep_geomad_test/0-0/test/2023/dep_geomad_test_test_2023.stac-item.json")